In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:14 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu 

In [2]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-02-26 21:49:34--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.50MB/s    in 0.2s    

2023-02-26 21:49:34 (4.50 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [4]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Mobile_Apps_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Mobile_Apps_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    1774101|R3PMQXEVYR4ZT1|B00DVKOYBM|     263046246|         8 Ball Pool|     Mobile_Apps|          3|            0|          0|   N|                Y|                 Fun|It is a fun game ...| 2015-08-31|
|         US|   13679234|R1I1K3EO98EMUV|B00R1IA4RS|     128868854|Christmas Cookie ...|     Mobile_Apps|          5|    

In [5]:
# Get the number of rows in the DataFrame.
df.count()

5033376

# Transform the Data

## Create the "review_id_table".

In [6]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
review_id_table = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3PMQXEVYR4ZT1|    1774101|B00DVKOYBM|     263046246| 2015-08-31|
|R1I1K3EO98EMUV|   13679234|B00R1IA4RS|     128868854| 2015-08-31|
|R3K23URSI2MY42|   15056632|B01069LGHK|     955481764| 2015-08-31|
| RH2P770T10X38|   52477711|B00X64ZKM0|     455029830| 2015-08-31|
| RV9YBAW8CWFYN|   15021042|B00KIZO238|     685358159| 2015-08-31|
| R19627K7G1G6X|    5181462|B00H9FE0FU|     158085302| 2015-08-31|
|R22858XECEF7YQ|   19360265|B0064X7FVE|     140639925| 2015-08-31|
|R3KQOY7MDZOK0U|   28928572|B00N28818A|     838378762| 2015-08-31|
|R31BKIN02QFDHS|   26842555|B00HGMXSGI|     141073643| 2015-08-31|
|R3FW62P5EUDUUV|   25535583|B0119R2DB6|     498391460| 2015-08-31|
| RBD8ROQJQO68E|    3343825|B00O0OR65E|     156086007| 2015-08-31|
|R1S04R7IT12FKH|   49181739|B00YCL9R0W|     629058532| 2015-08

## Create the "products" Table

In [7]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = df.select(["product_id", "product_title"]).drop_duplicates(["product_id", "product_title"])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00NG6CIKA|Solitaire TriPeak...|
|B00ZDVV7Q2|    Frozen Free Fall|
|B00N0RK9JM|How Draw Adventur...|
|B00SYZDSXO|       Bee Brilliant|
|B00HAPRVWS|Sonic The Hedgehog 2|
|B00UQK19Z4|HD Screen Wallpap...|
|B0073VRX5Y|Power-Button Flas...|
|B00ZHDCTLS|           bunny car|
|B00I3MX20W|            Vanguard|
|B00ITR6WLW|How to Draw: Eque...|
|B00CREXAVS|           Bridge V+|
|B00CSQYYHO|     Where's My Cat?|
|B009KVP70S|          War Sounds|
|B00KQ1XYFQ|   My Verizon Mobile|
|B014GD3NRS|          kumo cloud|
|B00EDUTGB2|     HP Print Plugin|
|B00NXOLPFY|Horse Simulator 3...|
|B004AHBBPW|         Daily Bible|
|B008I43DYU|      Calculator Pro|
|B00RG80MSY|        Jewels Blitz|
+----------+--------------------+
only showing top 20 rows



## Create the "customers" Table

In [8]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
customers_df = df.groupBy("customer_id").count()
customers_df.orderBy("customer_id").select(["customer_id", "count"])
customers_df.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   44552694|    7|
|   27457155|    6|
|   35842367|    9|
|   16848166|    5|
|   13313689|   10|
|    3628854|    1|
|   43626894|    1|
|    4658983|    8|
|   16457323|    3|
|   30024726|    5|
|    3636174|    1|
|     582750|    1|
|      78113|    1|
|    2246607|    5|
|   26264517|    1|
|    7439900|    1|
|    2361059|    2|
|   22965400|    2|
|   32777038|    1|
|   29019526|    2|
+-----------+-----+
only showing top 20 rows



## Create the "vine_table".

In [9]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3PMQXEVYR4ZT1|          3|            0|          0|   N|
|R1I1K3EO98EMUV|          5|            0|          0|   N|
|R3K23URSI2MY42|          5|            4|          4|   N|
| RH2P770T10X38|          5|            0|          0|   N|
| RV9YBAW8CWFYN|          1|            0|          0|   N|
| R19627K7G1G6X|          5|            0|          0|   N|
|R22858XECEF7YQ|          3|            0|          0|   N|
|R3KQOY7MDZOK0U|          1|            0|          0|   N|
|R31BKIN02QFDHS|          4|            0|          0|   N|
|R3FW62P5EUDUUV|          5|            0|          0|   N|
| RBD8ROQJQO68E|          5|            0|          0|   N|
|R1S04R7IT12FKH|          4|            0|          0|   N|
|R194G91R3ZLBF2|          1|            0|          0|   N|
|R3HL9X6I540JIU|          4|            

# Load

In [10]:
mode = "append"
jdbc_url="jdbc:postgresql://database-homework-22.ckurgpy6lirg.us-east-2.rds.amazonaws.com:5432/database_hw_22"
config = {"user": "<username_here>", "password": "<password_here>", "driver":"org.postgresql.Driver"}

In [12]:
# Write review_id_df to table in RDS
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table_mobileapps', mode=mode, properties=config)

In [13]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products_mobileapps', mode=mode, properties=config)

In [14]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers_mobileapps', mode=mode, properties=config)

In [15]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_mobileapps', mode=mode, properties=config)